In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np

%load_ext autoreload
%autoreload 2
from data_loader import CharacterDataset

In [9]:
PATH='./telegram_fetch/texts/maria_1ua'
PATH='./telegram_fetch/texts/masha_small'
#PATH='./telegram_fetch/texts/elvina'

In [10]:
dataset = CharacterDataset(PATH)

data shape: (55303, 163)


In [11]:
x = dataset[0]
N = x.shape[-1]
print(f"Dimension size {N}")
print(f"Dataset length {len(dataset)}")

Dimension size 163
Dataset length 55303


In [155]:
class Encoder(nn.Module):
    def __init__(self, inp, out):
        super().__init__()
        self.inp = inp
        self.out = out
        N_layers = 2
        ratio = np.power(inp/out, 1/N_layers)
        mult = int(ratio*out)
        print(mult)
        
        self.dense1 = nn.Linear(self.inp, mult )
        self.bn1= nn.BatchNorm1d(mult)
        self.dense2 = nn.Linear(mult, out )
        
    def forward(self,x):
        x = F.relu(self.dense1(x))
        #x = self.bn1(x)
        x = self.dense2(x)
        return x
    
class RNN(nn.Module):
    def __init__(self, inp_d, hidden=300):
        super().__init__()
        self.enc = Encoder(inp=inp_d+hidden, out=hidden)
        self.dec = Encoder(out=inp_d, inp = hidden)
    def forward(self, x):
        x = self.enc(x)
        x = self.dec(x)
        x = F.softmax(x,dim=-1)
        return x


In [156]:
HIDDEN = 20
rnn = RNN(N, hidden=HIDDEN)

def param_cnt(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"parameters count {param_cnt(rnn)}")
rnn

60
57
parameters count 23145


RNN(
  (enc): Encoder(
    (dense1): Linear(in_features=183, out_features=60, bias=True)
    (bn1): BatchNorm1d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dense2): Linear(in_features=60, out_features=20, bias=True)
  )
  (dec): Encoder(
    (dense1): Linear(in_features=20, out_features=57, bias=True)
    (bn1): BatchNorm1d(57, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dense2): Linear(in_features=57, out_features=163, bias=True)
  )
)

In [144]:
import torch.optim as optim

criterion = nn.BCELoss()
opt = optim.SGD(rnn.parameters(), lr = 0.01, momentum=.99)

In [145]:
torch.cuda.is_available()

True

In [146]:
torch.cuda.set_device(0)
rnn.cuda()

RuntimeError: CUDA error: unspecified launch failure

In [157]:
import time
BATCH = 1
dataloader = DataLoader(dataset, batch_size=BATCH)

In [158]:
start = time.time()
prev_output = torch.Tensor(np.zeros((BATCH,HIDDEN)))
char = torch.Tensor(np.zeros((BATCH,N)))

for epoch in range(10):
    running_loss = 0.0
    for i, data in (enumerate(dataloader,0)):
        next_char = data
        next_char = torch.Tensor(next_char.float())
        
        opt.zero_grad()
        """
        char = char.cuda()
        next_char = next_char.cuda()
        prev_output = prev_output.cuda()
        """
        inp = torch.cat((prev_output[:len(char)], char),1)
        coding = rnn.enc(inp)
        outputs = rnn.dec(coding)
        outputs = F.softmax(outputs,dim=-1)
        
        
        loss = criterion(outputs, next_char)
        loss.backward()
        opt.step()
        
        char = next_char.detach()
        prev_output = coding.detach()
        running_loss += loss.item()
        if i % 20000 == 0:
            delta_t = time.time()-start
            print(f"[{epoch}, {i}] {np.round(delta_t,3)} loss: {running_loss/20000}")
            start = time.time()
            running_loss = 0.0
        

[0, 0] 0.002 loss: 1.8698612228035927e-06
[0, 20000] 16.898 loss: 0.0375356404254213
[0, 40000] 17.455 loss: 0.03751989866103977
[1, 0] 15.05 loss: 1.8673975020647048e-06
[1, 20000] 19.467 loss: 0.03753564047794789
[1, 40000] 17.633 loss: 0.03751989866103977
[2, 0] 13.085 loss: 1.8673975020647048e-06
[2, 20000] 16.328 loss: 0.03753564047794789
[2, 40000] 17.797 loss: 0.03751989866103977
[3, 0] 14.209 loss: 1.8673975020647048e-06
[3, 20000] 17.329 loss: 0.03753564047794789
[3, 40000] 17.403 loss: 0.03751989866103977
[4, 0] 13.391 loss: 1.8673975020647048e-06


KeyboardInterrupt: 

In [139]:
inital_char = '<'
prev_char = dataset._encoder.transform([[inital_char]]).todense()
inital_char = 'B'
curr_char = dataset._encoder.transform([[inital_char]]).todense()
curr_char= torch.Tensor(curr_char)
prev_char= torch.Tensor(prev_char)

for i in range(25):
    inp = torch.cat((prev_char, curr_char),1)
    char = rnn(inp)
    decoded_char = dataset._encoder.inverse_transform(char.detach())
    prev_char = curr_char
    curr_char = torch.Tensor(dataset._encoder.transform(decoded_char).todense())
    #curr_char = char
    print(decoded_char, np.max(curr_char.detach().numpy()),np.sum(curr_char.detach().numpy()))
    

[['и']] 1.0 1.0
[[' ']] 1.0 1.0
[['<']] 1.0 1.0
[['M']] 1.0 1.0
[['S']] 1.0 1.0
[['E']] 1.0 1.0
[['R']] 1.0 1.0
[['>']] 1.0 1.0
[['\n']] 1.0 1.0
[['"']] 1.0 1.0
[['о']] 1.0 1.0
[['б']] 1.0 1.0
[[' ']] 1.0 1.0
[['с']] 1.0 1.0
[['н']] 1.0 1.0
[['о']] 1.0 1.0
[['в']] 1.0 1.0
[['у']] 1.0 1.0
[['с']] 1.0 1.0
[['м']] 1.0 1.0
[[' ']] 1.0 1.0
[['с']] 1.0 1.0
[['н']] 1.0 1.0
[['о']] 1.0 1.0
[['в']] 1.0 1.0
